In [1]:
pip install xgboost==2.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 5.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests

In [3]:
def get_data(group_number):
    url = f"http://10.43.101.149/data?group_number={group_number}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        # g = data["group_number"]
        # b = data["batch_number"]
        # print(g,b)
        # data = data["data"]
        # df = pd.DataFrame(data)
        # df.columns = ['brokered_by','status','price','bed','bath','acre_lot','street','city','state',
        #               'zip_code','house_size','prev_sold_date']
        return data
    else:
        print(f"Error retrieving data: {response.status_code}")
        return None

In [4]:
group_number = 2
data_p1 = get_data(group_number)

In [5]:
import pandas as pd
import os
import mlflow

In [6]:
def predic_model_train(data_predict):


    # conectar con mlflow y minio
    mlflow.set_tracking_uri("http://Mlflow:5000")

    os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://Minio:9000"
    os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
    os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'


    model_name = "tracking-price-XGBoost"
    model_version = 1

    lr = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

    return lr.predict(data_predict)


user_input = [3.0, 2.0, 0.09, 10.0, 1409.0]
columns = ['bed','bath','acre_lot', 'state', 'house_size']
df_pred = pd.DataFrame([user_input], columns=columns)
out_model = predic_model_train(df_pred)
out_model

2024/06/02 00:07:05 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 3.0.0, required: cloudpickle==2.2.1)
 - psutil (current: 5.9.8, required: psutil==5.9.5)
 - scikit-learn (current: 1.5.0, required: scikit-learn==0.24.2)
 - xgboost (current: 2.0.3, required: xgboost==1.6.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024/06/02 00:07:05 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.7.16`, differs from the version of Python that is currently running, `Python 3.9.19`, and may be incompatible
/usr/local/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:07:05] WARNING: /workspace/src/common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configurati

array([203584.98], dtype=float32)

In [7]:
import mlflow

client = mlflow.tracking.MlflowClient()

def print_all_registered_models():
    for rm in client.search_registered_models():
        print(f"Registered Model: {rm.name}")
        for mv in client.get_model_versions(rm.name):
            print(f"  Version: {mv.version}")

print_all_registered_models()

Registered Model: modelo_test


AttributeError: 'MlflowClient' object has no attribute 'get_model_versions'